# 兰州天气数据标准化分析

通过标准化处理，我们可以：
1. 消除不同量纲的影响
2. 更好地比较不同特征之间的关系
3. 发现数据的异常模式

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pyecharts import options as opts
from pyecharts.charts import Line, Scatter, HeatMap, Grid
from pyecharts.globals import ThemeType
import re

# 设置随机种子
np.random.seed(42)

In [6]:
# 数据预处理
def extract_temperature(temp_str):
    return int(temp_str.replace('℃', ''))

def extract_wind_force(wind_str):
    match = re.search(r'(\d+)-(\d+)级', wind_str)
    if match:
        return (int(match.group(1)) + int(match.group(2))) / 2
    return np.nan

# 读取数据
df = pd.read_excel('../数据集/lanzhou_weather_data.xlsx')

# 处理日期
df['日期'] = pd.to_datetime(df['日期'], format='%Y年%m月%d日')
df['年'] = df['日期'].dt.year
df['月'] = df['日期'].dt.month
df['季节'] = pd.cut(df['月'], bins=[0,3,6,9,12], labels=['春季','夏季','秋季','冬季'])

# 处理温度和风力
df['最高气温_数值'] = df['最高气温'].apply(extract_temperature)
df['最低气温_数值'] = df['最低气温'].apply(extract_temperature)
df['日温差'] = df['最高气温_数值'] - df['最低气温_数值']
df['白天风力等级'] = df['白天风力'].apply(extract_wind_force)
df['夜间风力等级'] = df['夜间风力'].apply(extract_wind_force)

## 1. Z-score标准化分析

In [37]:
# 选择需要标准化的数值特征
numeric_features = ['最高气温_数值', '最低气温_数值', '日温差', '白天风力等级', '夜间风力等级']

# 使用StandardScaler进行Z-score标准化
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df[numeric_features]),
    columns=[f"{col}_标准化" for col in numeric_features],
    index=df.index
)

# 计算标准化后的统计信息
print("标准化后的统计信息：")
print(df_scaled.describe().round(3))

# 可视化标准化结果
def plot_standardized_features():
    line = Line(init_opts=opts.InitOpts(theme=ThemeType.MACARONS))
    line.add_xaxis(df['日期'].dt.strftime('%Y-%m').unique().tolist())
    
    for col in df_scaled.columns:
        monthly_mean = df_scaled.groupby(df['日期'].dt.strftime('%Y-%m'))[col].mean()
        line.add_yaxis(
            col,
            monthly_mean.round(3).tolist(),
            is_symbol_show=False
        )
    
    line.set_global_opts(
        title_opts=opts.TitleOpts(title="标准化后的特征变化趋势"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[opts.DataZoomOpts()],
        yaxis_opts=opts.AxisOpts(name="标准化值")
    )
    return line.render('可视化图表/Z-score标准化分析.html')

plot_standardized_features()

标准化后的统计信息：
       最高气温_数值_标准化  最低气温_数值_标准化   日温差_标准化  白天风力等级_标准化  夜间风力等级_标准化
count     2197.000     2197.000  2197.000    2193.000    2193.000
mean        -0.000        0.000     0.000       0.000       0.000
std          1.000        1.000     1.000       1.000       1.000
min         -2.396       -2.210    -3.079      -0.457      -0.455
25%         -0.892       -0.993    -0.777      -0.457      -0.455
50%          0.048        0.123     0.086      -0.457      -0.455
75%          0.894        0.934     0.661       0.623       0.652
max          1.928        1.948     3.539       6.024       6.185


'E:\\jupyterNotebook\\redata\\数据分析项目\\数据分析\\可视化图表\\Z-score标准化分析.html'

## 2. 归一化分析（Min-Max标准化）

In [43]:
# 使用MinMaxScaler进行归一化
minmax_scaler = MinMaxScaler()
df_normalized = pd.DataFrame(
    minmax_scaler.fit_transform(df[numeric_features]),
    columns=[f"{col}_归一化" for col in numeric_features],
    index=df.index
)

# 可视化归一化结果
def plot_normalized_features():
    line = Line(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    line.add_xaxis(df['日期'].dt.strftime('%Y-%m').unique().tolist())
    
    for col in df_normalized.columns:
        monthly_mean = df_normalized.groupby(df['日期'].dt.strftime('%Y-%m'))[col].mean()
        line.add_yaxis(
            col,
            monthly_mean.round(3).tolist(),
            is_symbol_show=False
        )
    
    line.set_global_opts(
        title_opts=opts.TitleOpts(title="归一化后的特征变化趋势"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[opts.DataZoomOpts()],
        yaxis_opts=opts.AxisOpts(name="归一化值")
    )
    return line.render('可视化图表/Min_Max标准化分析.html')

plot_normalized_features()

'E:\\jupyterNotebook\\redata\\数据分析项目\\数据分析\\可视化图表\\Min_Max标准化分析.html'

## 3. 特征相关性分析

In [45]:
# 计算标准化后特征之间的相关性
correlation_matrix = df_scaled.corr()

# 可视化相关性热力图
def plot_correlation_heatmap():
    # 准备数据
    features = correlation_matrix.index.tolist()
    data = [
        [i, j, correlation_matrix.iloc[i, j]]
        for i in range(len(features))
        for j in range(len(features))
    ]
    
    heatmap = HeatMap(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
    heatmap.add_xaxis(features)
    heatmap.add_yaxis(
        "相关性",
        features,
        data,
        label_opts=opts.LabelOpts(is_show=True, formatter="{c}")
    )
    
    heatmap.set_global_opts(
        title_opts=opts.TitleOpts(title="标准化特征相关性热力图"),
        visualmap_opts=opts.VisualMapOpts(
            min_=-1,
            max_=1,
            is_calculable=True,
            orient="horizontal",
            pos_left="center"
        )
    )
    return heatmap.render('可视化图表/特征相关性分析.html')

plot_correlation_heatmap()

'E:\\jupyterNotebook\\redata\\数据分析项目\\数据分析\\可视化图表\\特征相关性分析.html'

## 4. 异常值检测

In [47]:
# 使用Z-score方法检测异常值
def detect_outliers(df_scaled, threshold=3):
    outliers = {}
    for col in df_scaled.columns:
        # 找出超过threshold个标准差的值
        outliers[col] = df_scaled[abs(df_scaled[col]) > threshold].index.tolist()
    return outliers

outliers = detect_outliers(df_scaled)

# 可视化异常值
def plot_outliers():
    scatter = Scatter(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC))
    
    # 选择最高气温和最低气温进行展示
    x_data = df_scaled['最高气温_数值_标准化'].tolist()
    y_data = df_scaled['最低气温_数值_标准化'].tolist()
    
    # 标记正常值
    normal_indices = set(range(len(x_data))) - set(outliers['最高气温_数值_标准化']) - set(outliers['最低气温_数值_标准化'])
    scatter.add_xaxis([x_data[i] for i in normal_indices])
    scatter.add_yaxis(
        "正常值",
        [y_data[i] for i in normal_indices],
        symbol_size=10
    )
    
    # 标记异常值
    outlier_indices = set(outliers['最高气温_数值_标准化']) | set(outliers['最低气温_数值_标准化'])
    if outlier_indices:
        scatter.add_xaxis([x_data[i] for i in outlier_indices])
        scatter.add_yaxis(
            "异常值",
            [y_data[i] for i in outlier_indices],
            symbol_size=20,
            itemstyle_opts=opts.ItemStyleOpts(color="red")
        )
    
    scatter.set_global_opts(
        title_opts=opts.TitleOpts(title="气温异常值检测"),
        xaxis_opts=opts.AxisOpts(name="标准化最高气温"),
        yaxis_opts=opts.AxisOpts(name="标准化最低气温")
    )
    return scatter.render('可视化图表/异常值检测分析.html')

plot_outliers()

'E:\\jupyterNotebook\\redata\\数据分析项目\\数据分析\\可视化图表\\异常值检测分析.html'

## 5. 标准化分析结论

通过标准化分析，我们可以得出以下结论：

1. 数据分布特征：
   - Z-score标准化后的数据均值为0，标准差为1
   - 归一化后的数据都被压缩到[0,1]区间

2. 特征相关性：
   - [根据相关性热力图填写具体发现]
   - [填写重要的相关性模式]

3. 异常值发现：
   - [填写发现的异常值情况]
   - [分析异常值的可能原因]

4. 数据质量建议：
   - [填写数据质量相关建议]
   - [填写数据处理建议]